<h1>Create React App Structure Using Multi Agents </h1>
<h3>Using only OpenAI to create an app structure for react native apps. </h3>
<h6>The files won't contain any source code yet, for this OpenAI SDK would be the best, to create Agents and create source code.</h6>

In [ ]:
# Let's import environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
import os
import json
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if not openai_api_key:
    print('Missing OpenaAI API key.')
if not deepseek_api_key:
    print('Missing Deepseek API key')
if openai_api_key and deepseek_api_key:
    print(f'OpenAI: {openai_api_key[-10:]}\n')
    print(f'Deepseek: {deepseek_api_key[-10:]}\n')


In [ ]:
openai = OpenAI()

In [ ]:

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [ ]:
# system prompt form entrepreneur want to create a SASS app.  
 
system_prompt = """
You're an entrepreneur looking to develop and invest in emerging AI-driven small SaaS applications, 
to solve huge numbers of users' pain point for small business, for example book keeping, reservations, taxes, employees records and so on. 
You focus on solutions that use agentic AI for solving real-world small business problems with minimal human oversight,
offering scalability and innovation. You select the the idea that that has the highest potential for market disruption at the same time,
helping small business to save time and money. 
"""

user_prompt = "Pick a business area that might be worth exploring for Agentic AI, Respond with just the business area name. "

messages = [
    {"role": "system", "content":system_prompt},
    {"role": "user", "content": user_prompt},
]

# Call openai
response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

# Then read the business idea:
business_idea = response.choices[0].message.content
display(Markdown(business_idea))

In [ ]:
# We'll store the structure of the app
app = []

In [ ]:
app.append(
    {"business_idea": business_idea}
)

In [ ]:
app

In [ ]:
# Create a second prompt for pain points in business idea category
user_second_prompt = f"Find user pain points in {business_idea} areas. Give reasons and why this pain point is the best to solve. List the top 1."
second_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_second_prompt}
]

In [ ]:
# Call openai for pain points 
response = openai.chat.completions.create(
    messages=second_messages,
    model="gpt-4.1-mini"
)

pain_point = response.choices[0].message.content
display(Markdown(pain_point))

In [ ]:
# Add pain points 
app.append({"pain_points": pain_point})

In [ ]:
app

In [ ]:
# Let's create an app structure, for website, since we have all the information. 
third_user_prompt =f"""Please create a react website file directory structure. You're given the business idea, along 
with the following pain points \n. {pain_point}.
Respond in json format. The structure should also include stripe payment, and a database.
Please, add as much details as possible for explaining directory and files/ components. 
For example:

    "payment.tsx" : "stripe payment integration endpoints, for visa, credit and any other cards. User from USA, CANADA, and EUROPE, can make easy monthly subscription. "
    "user.tsx" : "user authentication and management APIs with the following library Authlib, and with social logins API such as Google and Microsoft... "

"""

third_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": third_user_prompt}
]

response  = openai.chat.completions.create(
    messages=third_messages,
    model="gpt-4.1-mini"    
)
structure = response.choices[0].message.content
display(Markdown(structure))


In [ ]:
app.append({"app_structure": structure})

In [ ]:
app

In [ ]:
system_prompt = "You're a react app developer"

In [ ]:
double_check_app_structure_prompt = f"""You're given the following {app}, please double check 
if there are any errors with structure, or  navigation. If there're mistake please show them with an explanation.
If no errors, please jus say 'No need to modify' """

In [ ]:
messages = [
    {"role": "system", "content" : system_prompt},
    {"role": "user", "content": double_check_app_structure_prompt}
]

In [ ]:
"""
We need to double check if the app structure is correct. We can use other models, 
deepseek seems to add extra files, and stays out of context, so let's stick with 
openai for now. 
"""
response = openai.chat.completions.create(
    messages=messages,
    model="gpt-4.1"  
)

double_check = response.choices[0].message.content
display(Markdown(double_check))

In [ ]:
implement = False
if double_check == 'No need to modify':
    implement = True
else:
    implement = False


In [ ]:
# Create the file structure recursively, from structure in current directory
def create_file_structure(structure, parent_dir='.'):
    for name, content in structure.items():
        path = os.path.join(parent_dir, name)
        if isinstance(content, dict):
            # It's a directory
            os.makedirs(path, exist_ok=True)
            create_file_structure(content, path)
        else:
            # It's a file, create empty file
            os.makedirs(parent_dir, exist_ok=True)
            with open(path, 'w', encoding='utf-8') as f:
                # We can write the description as comment for reference, or leave empty
                # Here writing description as comment to the file
                if name.endswith(('.ts', '.tsx', '.js', '.jsx')):
                    # for typescript/js files, use empty lines
                    for line in content.split('\n'):
                        f.write(f"\n")
                elif name.endswith(('.css', '.json')):
                    # css uses /* */, json uses nothing usually, write as json comment-like
                    for line in content.split('\n'):
                        f.write(f"\n")
                elif name.endswith('.md'):
                    # markdown use >
                    for line in content.split('\n'):
                        f.write(f"\n")
                else:
                    # default write plain text description
                    f.write(content + '\n')

In [ ]:
if implement:
    print("Creating the app structure directory and files...")
    # Remove markdown code block if present
    if structure.startswith("```json"):
        json_str = structure.strip()[7:-3].strip()
    else:
        json_str = structure
        
    structure_dict = json.loads(json_str)
    create_file_structure(structure_dict)
    print("App structure created.")
else:
    print("App structure needs modification, not implementing yet.")
    
    # Let's call openai to fix the structure, if needed.
    # We can loop until the structure is correct, but for now let's stop here.
    # TODO

In [ ]:
response = openai.chat.completions.create(
    messages=[
        {"role":"system", "content": system_prompt},
        {"role": "user","content": function_prompt}
    ],
    model="gpt-4.1-mini"
)
dir_func = response.choices[0].message.content
display(Markdown(dir_func))

In [ ]:
system_prompt = f"""You're Senior react developer with over 10 years of experience. 
You always double check your work."""

user_prompt = f"""Please write the source code for src/components files only. You're given 
the following app details in the {structure}\n
You're also provided with tool to save the source code."""